In [16]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size=(2,4))
print(X)
net(X)

tensor([[0.7110, 0.0559, 0.3083, 0.9353],
        [0.3263, 0.7953, 0.5347, 0.4284]])


tensor([[-0.0366],
        [ 0.0493]], grad_fn=<AddmmBackward0>)

In [17]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0491,  0.0566,  0.0122,  0.2473, -0.1045,  0.0471,  0.1917, -0.1203]])), ('bias', tensor([-0.1467]))])


In [18]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1467], requires_grad=True)
tensor([-0.1467])


In [19]:
net[2].weight.grad == None

True

In [20]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [21]:
net.state_dict()['2.bias'].data

tensor([-0.1467])

In [38]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2166],
        [-0.2166]], grad_fn=<AddmmBackward0>)

In [39]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [40]:
rgnet[0][1][0].bias.data

tensor([ 0.2854,  0.4608, -0.0775, -0.4364,  0.1310,  0.2979,  0.2980,  0.0852])

In [41]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0044, -0.0049, -0.0032, -0.0100]), tensor(0.))

In [42]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [43]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.6102,  0.3121, -0.1905,  0.0820])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [44]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init",*[(name,param.shape)
                       for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 8.1374, -0.0000,  0.0000,  0.0000],
        [-5.2403, -5.9539,  0.0000, -0.0000]], grad_fn=<SliceBackward0>)

In [46]:
net[0].weight.data[:] += 1
net[0].weight.data[0,0] = 42
net[0].weight.data[0]

tensor([42.,  2.,  2.,  2.])

In [47]:
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                    shared,nn.ReLU(),
                    shared,nn.ReLU(),
                    nn.Linear(8,1))
net(X)
#检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
